In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/reviews_analysis')

In [4]:
import pandas as pd

In [5]:
def count_duplicates(df):
  return df.groupby(df.columns.tolist(), as_index= False ).size()

In [6]:
def filter_duplicates(df):
  ndf = count_duplicates(df)
  return ndf.drop(['size'], axis=1)

In [26]:
reviews = pd.read_csv('all_reviews.csv', encoding = 'utf-8')

In [27]:
reviews.head()

,id,date,stars,review_text
0,1276123110,2024-01-13 08:17:29.661,5.0,"У меня к озону, как к сервису, и конкретно к э..."
1,1276123110,2024-04-04 13:23:17.859,5.0,"Очень хороший пункт, удобно расположен, чисто,..."
2,1276123110,2023-01-01 12:19:35.464,4.0,"Пункт выдачи удобно расположен, очень легко на..."
3,1276123110,2023-05-09 10:59:03.823,5.0,"Все отлично, расположение, чисто, просторно, н..."
4,1276123110,2023-11-09 16:21:06.131,5.0,"хороший пункт, приятная работница, заказы полу..."


In [28]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12221 entries, 0 to 12220
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           12221 non-null  int64  
 1   date         12221 non-null  object 
 2   stars        12218 non-null  float64
 3   review_text  12219 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 382.0+ KB


In [29]:
def find_NaN(df):
  return df[df.isna().any(axis=1)]

In [30]:
find_NaN(reviews)

,id,date,stars,review_text
1708,25010521811,2024-03-26 16:45:23.734,5.0,NaN
4413,74794040308,2024-06-05 19:38:31.052,NaN,Здравствуйте. Хочу пожаловаться на выданный за...
5152,88497708729,2022-09-16 19:22:52.779,NaN,30 секунд и все что заказал получил... Ходить ...
10562,207911795530,2024-02-06 13:21:56.288,5.0,NaN
11248,221560574924,2021-06-26 17:28:32.749,NaN,"Небольшой, уютный"


In [9]:
!pip install dostoevsky

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227138 sha256=bb1c3940087c3233e6c94549464d935adc3d209376963988fb2336221e909be9
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [10]:
!python -m dostoevsky download fasttext-social-network-model

In [11]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

In [12]:
def get_sentiment_analysis(df):
  tokenizer = RegexTokenizer()
  model = FastTextSocialNetworkModel(tokenizer=tokenizer)

  results = model.predict(df['review_text'], k=2)
  keys = ['positive', 'neutral', 'negative']

  for key in keys:
    df[key] = [res.get(key) for res in results]

  df[keys] = df[keys].fillna(value=0)
  return df

In [36]:
filtered_reviews = reviews.dropna()
filtered_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12216 entries, 0 to 12220
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           12216 non-null  int64  
 1   date         12216 non-null  object 
 2   stars        12216 non-null  float64
 3   review_text  12216 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 477.2+ KB


In [37]:
sent_rev = get_sentiment_analysis(filtered_reviews)
sent_rev

<ipython-input-12-97c605d23c5d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = [res.get(key) for res in results]
<ipython-input-12-97c605d23c5d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = [res.get(key) for res in results]
<ipython-input-12-97c605d23c5d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

,id,date,stars,review_text,positive,neutral,negative
0,1276123110,2024-01-13 08:17:29.661,5.0,"У меня к озону, как к сервису, и конкретно к э...",0.000000,0.407343,0.314061
1,1276123110,2024-04-04 13:23:17.859,5.0,"Очень хороший пункт, удобно расположен, чисто,...",0.938134,0.000000,0.071601
2,1276123110,2023-01-01 12:19:35.464,4.0,"Пункт выдачи удобно расположен, очень легко на...",0.173298,0.245095,0.000000
3,1276123110,2023-05-09 10:59:03.823,5.0,"Все отлично, расположение, чисто, просторно, н...",0.422515,0.000000,0.177821
4,1276123110,2023-11-09 16:21:06.131,5.0,"хороший пункт, приятная работница, заказы полу...",0.777310,0.000000,0.000000
...,...,...,...,...,...,...,...
12216,245536924647,2022-05-03 05:31:04.866,5.0,Быстрое обслуживание,0.228166,0.644235,0.000000
12217,245536924647,2022-12-18 13:21:45.710,5.0,Всё по делу,0.000000,0.855861,0.000000
12218,245536924647,2021-10-05 16:58:36.684,5.0,Высокий уровень обслуживания,0.000000,0.771854,0.000000
12219,245536924647,2022-05-11 15:30:47.822,5.0,Спасибо Оzon,0.000000,0.000000,0.000000


In [39]:
sum(sent_rev['positive']*sent_rev['neutral']*sent_rev['negative'])

0.0

In [33]:
def get_mark(df):
  s = df['positive'] + df['neutral'] + df['negative']
  df['mark'] = (5*df['positive'] + 3*df['neutral'] + 1*df['negative'])/s
  return df

In [40]:
analysis = get_mark(sent_rev)
analysis

<ipython-input-33-cd74dc18892c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mark'] = (5*df['positive'] + 3*df['neutral'] + 1*df['negative'])/s


,id,date,stars,review_text,positive,neutral,negative,mark
0,1276123110,2024-01-13 08:17:29.661,5.0,"У меня к озону, как к сервису, и конкретно к э...",0.000000,0.407343,0.314061,2.129307
1,1276123110,2024-04-04 13:23:17.859,5.0,"Очень хороший пункт, удобно расположен, чисто,...",0.938134,0.000000,0.071601,4.716357
2,1276123110,2023-01-01 12:19:35.464,4.0,"Пункт выдачи удобно расположен, очень легко на...",0.173298,0.245095,0.000000,3.828399
3,1276123110,2023-05-09 10:59:03.823,5.0,"Все отлично, расположение, чисто, просторно, н...",0.422515,0.000000,0.177821,3.815190
4,1276123110,2023-11-09 16:21:06.131,5.0,"хороший пункт, приятная работница, заказы полу...",0.777310,0.000000,0.000000,5.000000
...,...,...,...,...,...,...,...,...
12216,245536924647,2022-05-03 05:31:04.866,5.0,Быстрое обслуживание,0.228166,0.644235,0.000000,3.523077
12217,245536924647,2022-12-18 13:21:45.710,5.0,Всё по делу,0.000000,0.855861,0.000000,3.000000
12218,245536924647,2021-10-05 16:58:36.684,5.0,Высокий уровень обслуживания,0.000000,0.771854,0.000000,3.000000
12219,245536924647,2022-05-11 15:30:47.822,5.0,Спасибо Оzon,0.000000,0.000000,0.000000,NaN


In [43]:
find_NaN(analysis)

,id,date,stars,review_text,positive,neutral,negative,mark
134,1347737832,2021-08-15 14:01:40.186,5.0,Все быстро и вежливо. Спасибо,0.0,0.0,0.0,NaN
137,1347737832,2021-05-24 22:03:38.485,5.0,спасибо Валерии за хорошее настроение при выда...,0.0,0.0,0.0,NaN
357,1804018955,2024-02-19 18:29:21.000,5.0,"Пункт выдачи хороший, удобный, персонал вежлив...",0.0,0.0,0.0,NaN
365,1804018955,2024-02-19 18:15:08.107,5.0,Сотрудница Дарья была очень приветлива и вежли...,0.0,0.0,0.0,NaN
388,1804018955,2019-07-02 08:58:37.852,5.0,"Обычный пункт выдачи, приветливый персонал",0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...
11197,220832692068,2022-11-07 19:04:14.680,5.0,"Вежливы,оперативны! Спасибо",0.0,0.0,0.0,NaN
11308,224293156036,2023-04-24 16:58:52.092,5.0,"Быстро, чисто, вежливые сотрудники! Спасибо))",0.0,0.0,0.0,NaN
11625,232426543164,2021-02-02 15:22:58.074,5.0,"Спасибо, что вы есть возле дома!",0.0,0.0,0.0,NaN
12075,241806189646,2021-12-30 21:44:23.137,5.0,"Отлично! Быстро, четко, а самое главное, в сос...",0.0,0.0,0.0,NaN


In [23]:
from sklearn.metrics import mean_absolute_error

In [42]:
final = analysis.dropna()
mean_absolute_error(final['stars'], final['mark'])

1.2956939885080108